In [2]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [6]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 3.965444564819336, Val Loss - 2.3833744525909424, Val Accuracy - 0.404
Epoch 20, Train Loss - 0.4417363107204437, Val Loss - 1.2137351036071777, Val Accuracy - 0.726
Epoch 40, Train Loss - 0.22685450315475464, Val Loss - 1.5566155910491943, Val Accuracy - 0.716
Epoch 60, Train Loss - 0.0982513576745987, Val Loss - 1.8955780267715454, Val Accuracy - 0.715
Epoch 80, Train Loss - 0.13526694476604462, Val Loss - 2.1789731979370117, Val Accuracy - 0.737
Epoch 100, Train Loss - 0.0842047780752182, Val Loss - 2.122013568878174, Val Accuracy - 0.736
Epoch 120, Train Loss - 0.08908068388700485, Val Loss - 2.564542531967163, Val Accuracy - 0.72
Epoch 140, Train Loss - 0.06388646364212036, Val Loss - 2.2465193271636963, Val Accuracy - 0.749
Epoch 160, Train Loss - 0.05506541579961777, Val Loss - 2.474992275238037, Val Accuracy - 0.741
Epoch 180, Train Loss - 0.0760008916258812, Val Loss - 2.5688796043395996, Val Accuracy - 0.739
Epoch 200, Train Loss - 0.08071742206811905, V

In [7]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.738
Initial Loss: 2.5238640308380127


In [8]:
amts = defaultdict(int)
# Run 10 attacks for perturbation of 0.2
for _ in range(10):
    attacker = Metattack(data, device=device)
    attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask, lambda_=0.)
    attacker.reset()
    attacker.attack(0.2)

    for edge in attacker._added_edges.keys():
        amts[edge] += 1


Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

In [9]:
amts

defaultdict(int,
            {(21, 2410): 10,
             (49, 2432): 10,
             (57, 2411): 10,
             (7, 2603): 10,
             (86, 2255): 10,
             (4, 2539): 10,
             (38, 259): 10,
             (101, 355): 10,
             (21, 959): 10,
             (131, 2210): 10,
             (115, 187): 10,
             (131, 2504): 10,
             (10, 2612): 10,
             (62, 1347): 10,
             (29, 1596): 10,
             (29, 2697): 10,
             (13, 500): 10,
             (27, 1591): 10,
             (79, 2541): 10,
             (35, 1615): 10,
             (60, 2609): 10,
             (76, 2513): 10,
             (50, 2486): 10,
             (50, 569): 10,
             (23, 1310): 10,
             (120, 1084): 10,
             (37, 846): 10,
             (12, 47): 10,
             (115, 2545): 10,
             (10, 1547): 10,
             (79, 1684): 10,
             (98, 2163): 10,
             (54, 2545): 10,
             (29, 1612): 10,
  

In [10]:
sorted_list = sorted(amts.items(), key=lambda item: item[1], reverse=True)
sorted_keys = [key for key, value in sorted_list]

In [11]:
edges_to_add = sorted_keys

In [12]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [13]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.15
budget = int(ptb_rate * initial_edge_count)

In [14]:
print(len(edges_to_add))
print(budget)

962
395


In [15]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_to_add):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_to_add[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [16]:
splits = [0, 0.5, 0.7, 0.9]
split_dic = defaultdict(list)

In [17]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.0750
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/395 [00:00<?, ?it/s]

Final Accuracy: 0.7260
Accuracy Drop: 0.0120
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0375
Phase 1: Added 197 edges out of requested 197.


Peturbing graph...:   0%|          | 0/205 [00:00<?, ?it/s]

Final Accuracy: 0.7260
Accuracy Drop: 0.0120
0.7

--- Running split: 0.7 ---
Second phase perturbation rate: 0.0225
Phase 1: Added 276 edges out of requested 276.


Peturbing graph...:   0%|          | 0/124 [00:00<?, ?it/s]

Final Accuracy: 0.7310
Accuracy Drop: 0.0070
0.9

--- Running split: 0.9 ---
Second phase perturbation rate: 0.0075
Phase 1: Added 355 edges out of requested 355.


Peturbing graph...:   0%|          | 0/42 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 56.00 MiB (GPU 0; 31.74 GiB total capacity; 528.77 MiB already allocated; 55.12 MiB free; 620.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.5
budget = int(ptb_rate * initial_edge_count)

In [20]:
print(len(edges_to_add))
print(budget)

724
1319
